In [1]:
from typing import Callable

import polars as pl
from faker import Faker

fake = Faker()

%load_ext autoreload
%autoreload 2

In [2]:
datapath = "/home/grantham/windmark/data/ledgers/trxns.csv"

df = pl.read_csv(datapath).filter(pl.col("User") < 500)

In [3]:
df.head()

User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
i64,i64,i64,i64,i64,str,str,str,i64,str,str,f64,i64,str,str
0,0,2002,9,1,"""06:21""","""$134.09""","""Swipe Transact…",3527213246127876953,"""La Verne""","""CA""",91750.0,5300,null,"""No"""
0,0,2002,9,1,"""06:42""","""$38.48""","""Swipe Transact…",-727612092139916043,"""Monterey Park""","""CA""",91754.0,5411,null,"""No"""
0,0,2002,9,2,"""06:22""","""$120.34""","""Swipe Transact…",-727612092139916043,"""Monterey Park""","""CA""",91754.0,5411,null,"""No"""
0,0,2002,9,2,"""17:45""","""$128.95""","""Swipe Transact…",3414527459579106770,"""Monterey Park""","""CA""",91754.0,5651,null,"""No"""
0,0,2002,9,3,"""06:23""","""$104.71""","""Swipe Transact…",5817218446178736267,"""La Verne""","""CA""",91750.0,5912,null,"""No"""


In [4]:
df.get_column("Use Chip").unique()

Use Chip
str
"""Chip Transacti…"
"""Swipe Transact…"
"""Online Transac…"


In [5]:
errors = (
    pl.read_csv(datapath)
    .get_column("Errors?")
    .drop_nulls()
    .unique()
    .to_list()
)

unique_errors = set("".join(errors).split(","))

unique_errors.remove("")

print(unique_errors)

{'Bad Expiration', 'Bad Zipcode', 'Technical Glitch', 'Bad CVV', 'Bad Card Number', 'Bad PIN', 'Insufficient Balance'}


In [6]:
df = pl.read_csv(datapath).filter(pl.col("User") < 500)

df.columns = [col.lower().replace("?", "").replace(" ", "_") for col in df.columns]

def mock(dataframe: pl.DataFrame, name: str, generator: Callable = fake.unique.company):
    values: list[str] = dataframe.get_column(name).cast(pl.String).unique().to_list()

    mapping = {value: generator() for value in values}

    return dataframe.select(pl.exclude(name), pl.col(name).replace(mapping).alias(name))


df = mock(dataframe=df, name="merchant_name", generator=fake.unique.company)
df = mock(dataframe=df, name="card", generator=fake.unique.credit_card_number)
df = mock(dataframe=df, name="user", generator=fake.unique.name)

ledger = (
    df
    # .filter(pl.col("user") < 100)
    .select(pl.all(), timeparts=pl.col("time").str.split(":"))
    .select(
        pl.all(),
        hour=pl.col("timeparts").list.first(),
        minute=pl.col("timeparts").list.last(),
    )
    .select(
        pl.all(),
        timestamp=pl.datetime(pl.col("year"), pl.col("month"), pl.col("day"), pl.col("hour"), pl.col("minute")),
    )
    .select(
        "use_chip",
        "merchant_state",
        "merchant_city",
        pl.col("mcc").cast(pl.String),
        "card",
        "timestamp",
        *[
            pl.col("errors").fill_null("").str.contains(error).cast(pl.String).alias(f'has_{error.lower().replace(" ", "_")}')
            for error in unique_errors
        ],
        amount=pl.col("amount").str.strip_prefix("$").str.to_decimal(),
        merchant_name=pl.col("merchant_name").cast(pl.Utf8),
        is_fraud=pl.col("is_fraud"),
        transaction_id=pl.col("timestamp").cast(pl.Utf8),
        customer_id=pl.col("user"),
    )
    .sort("timestamp")
    .select(
        pl.all(),
        order_id=pl.col("transaction_id").cum_count().over("customer_id"),
        timedelta=pl.col("timestamp").sub(pl.col("timestamp").shift().over("customer_id")).dt.total_seconds(),
        tenure=pl.col("timestamp").sub(pl.col("timestamp").first().over("customer_id")).dt.total_seconds(),
    )
)

In [7]:
ledger.head()

use_chip,merchant_state,merchant_city,mcc,card,timestamp,has_bad_expiration,has_bad_zipcode,has_technical_glitch,has_bad_cvv,has_bad_card_number,has_bad_pin,has_insufficient_balance,amount,merchant_name,is_fraud,transaction_id,customer_id,order_id,timedelta,tenure
str,str,str,str,str,datetime[μs],str,str,str,str,str,str,str,"decimal[*,2]",str,str,str,str,u32,i64,i64
"""Swipe Transact…","""CA""","""Sacramento""","""3058""","""38210207079055…",1991-11-25 06:55:00,"""false""","""false""","""false""","""false""","""false""","""false""","""false""",355.71,"""Pena, Hughes a…","""No""","""1991-11-25 06:…","""Albert Hill""",1,null,0
"""Swipe Transact…","""CA""","""San Jose""","""4829""","""38210207079055…",1991-12-01 06:51:00,"""false""","""false""","""false""","""false""","""false""","""false""","""false""",100,"""Hansen-Rose""","""No""","""1991-12-01 06:…","""Albert Hill""",2,518160,518160
"""Swipe Transact…","""CA""","""Watsonville""","""5411""","""38210207079055…",1991-12-01 09:36:00,"""false""","""false""","""false""","""false""","""false""","""false""","""false""",2.10,"""Walsh-Torres""","""No""","""1991-12-01 09:…","""Albert Hill""",3,9900,528060
"""Swipe Transact…","""CA""","""Watsonville""","""5411""","""38210207079055…",1991-12-01 09:44:00,"""false""","""false""","""false""","""false""","""false""","""false""","""false""",48.93,"""Lewis-Jones""","""No""","""1991-12-01 09:…","""Albert Hill""",4,480,528540
"""Swipe Transact…","""CA""","""Watsonville""","""5912""","""38210207079055…",1991-12-01 10:21:00,"""false""","""false""","""false""","""false""","""false""","""false""","""false""",52.23,"""Hanna Inc""","""No""","""1991-12-01 10:…","""Albert Hill""",5,2220,530760


In [8]:
from zlib import crc32

def assign_split(column: str) -> pl.Expr:

    seed = (
        pl.col(column)
        .cast(pl.String)
        .map_elements(lambda x: float(crc32(str.encode(x)) & 0xFFFFFFFF), return_dtype=pl.Float32)
        .mul(1 / 2**32)
    )

    return (
        pl.when(seed.is_between(0.0, 0.6))
        .then(pl.lit("train"))
        .when(seed.is_between(0.6, 0.8))
        .then(pl.lit("validate"))
        .when(seed.is_between(0.8, 1.0))
        .then(pl.lit("test"))
        .otherwise(pl.lit("train"))
    )

In [9]:
fields = [
    'use_chip',
    'merchant_state',
    'merchant_city',
    'mcc',
    'card',
    'timestamp',
    'has_technical_glitch',
    'has_bad_pin',
    'has_bad_zipcode',
    'has_insufficient_balance',
    'has_bad_cvv',
    'has_bad_card_number',
    'has_bad_expiration',
    'amount',
    'merchant_name',
    # 'is_fraud',
    # 'transaction_id',
    # 'customer_id',
    # 'order_id',
    'timedelta',
    'tenure',
]

In [10]:
lifestreams = (
    ledger.select(
        *[field for field in fields],
        'is_fraud',
        'transaction_id',
        'customer_id',
        'order_id',
        split=assign_split('customer_id'),
    )
    .sort("customer_id", "order_id")
    .group_by("customer_id", maintain_order=True)
    .agg(
        *[field for field in fields],
        'transaction_id',
        size=pl.len().cast(pl.Int32),
        is_fraud=pl.col('is_fraud'),
        split=pl.col("split").last(),
    )
    .iter_slices(5)
)

In [12]:
for index, lifestream in enumerate(lifestreams):
    lifestream.write_avro(f"../data/lifestreams/trxns/trxns-{index}.avro", name="trxns")